In [258]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import copy
from math import sqrt,ceil

from clyent import color
from scipy.stats import linregress
import libraries as lib

In [259]:
directory = "../../HvA/CleanMobility - E-team/Logfiles/H2A/2022/20220628-20220701_race_nogaro/0023 - 20220701T141014 1h8m8s 13.336km Nogaro/"
file = "1.csv"

In [260]:
data_file = lib.DataInladen(f'{directory}{file}')

In [261]:
data_file

,Datalogger port,"Dataloggertijd, in s",Format header (>08|03),NMEA type($GPRMC),Tijd,Status,Latitude,Noord of Zuid (N/S),Longitude,Oost of West (E/W),...,Mode indicator (A),Data in balancers,DegreesTrue,T,DegreesMagnetic,M,Snelheid tenopzichte van het water in Knots,N,Snelheid tenopzichte van het water in Kilometers,K
0,1,70.8626,08|03,$GPRMC,120813.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,71.8578,08|03,$GPRMC,120814.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,72.8577,08|03,$GPRMC,120815.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,73.8609,08|03,$GPRMC,120816.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,74.8585,08|03,$GPRMC,120817.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,1,4276.0450,08|03,$GPRMC,131818.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4201,1,4277.0633,08|03,$GPRMC,131819.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4202,1,4278.0288,08|03,$GPRMC,131820.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4203,1,4279.0253,08|03,$GPRMC,131821.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [262]:
yas = 'Latitude'
xas = 'Longitude'
colorbar = 'Snelheid over de grond in km/h'
# Filtering the data so it's only the laps we want to analyze
# First filter so that the data without GPS data is removed
data_file = data_file[data_file[xas] != 0]
data_file = data_file[data_file[yas] != 0]
data_file = data_file[data_file[colorbar] != 0]


# Now filter out the beginning and end of the data, by removing the first and last 10% of the data
from math import ceil
filter = data_file.iloc[ceil(len(data_file)*0):ceil(len(data_file)*1)]
filter

,Datalogger port,"Dataloggertijd, in s",Format header (>08|03),NMEA type($GPRMC),Tijd,Status,Latitude,Noord of Zuid (N/S),Longitude,Oost of West (E/W),...,Mode indicator (A),Data in balancers,DegreesTrue,T,DegreesMagnetic,M,Snelheid tenopzichte van het water in Knots,N,Snelheid tenopzichte van het water in Kilometers,K
124,1,194.9040,08|03,$GPRMC,121017.0,A,4346.2889,N,2.5416,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,1,195.9052,08|03,$GPRMC,121018.0,A,4346.2857,N,2.5432,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,1,196.9250,08|03,$GPRMC,121019.0,A,4346.2846,N,2.5439,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,1,197.9082,08|03,$GPRMC,121020.0,A,4346.2842,N,2.5435,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,1,198.9123,08|03,$GPRMC,121021.0,A,4346.2840,N,2.5428,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4047,1,4123.0314,08|03,$GPRMC,131545.0,A,4346.3116,N,2.5435,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4048,1,4124.0539,08|03,$GPRMC,131546.0,A,4346.3121,N,2.5436,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4052,1,4128.0340,08|03,$GPRMC,131550.0,A,4346.3122,N,2.5434,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4060,1,4136.0484,08|03,$GPRMC,131558.0,A,4346.3124,N,2.5425,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [278]:


lat_min, lat_max = 2.445, 2.455
lon_min, lon_max = 0.28+4.346e3,0.305+4.346e3 
yas = 'Latitude'
xas = 'Longitude'

# Itereer over elke rij in de DataFrame
for i in range(len(filter)):
    lat = filter[xas].iloc[i]  # Latitude van de i-de rij
    lon = filter[yas].iloc[i]  # Longitude van de i-de rij
    
    # Controleer of het punt binnen het vierkant valt
    if lat_min <= lat <= lat_max:
        #print(f"rij {i}: top")
        if lon_min <= lon <= lon_max:
            print(f"Rij {i}: The point is inside the square.")


Rij 509: The point is inside the square.
Rij 510: The point is inside the square.
Rij 717: The point is inside the square.
Rij 718: The point is inside the square.
Rij 985: The point is inside the square.
Rij 986: The point is inside the square.
Rij 1194: The point is inside the square.
Rij 1195: The point is inside the square.
Rij 1417: The point is inside the square.
Rij 1418: The point is inside the square.
Rij 1635: The point is inside the square.
Rij 1636: The point is inside the square.
Rij 1901: The point is inside the square.
Rij 1902: The point is inside the square.
Rij 1903: The point is inside the square.
Rij 1904: The point is inside the square.


In [272]:
§§§§§§§# Assuming 'filter' is your DataFrame and 'xas', 'yas', 'colorbar' are the column names
plt.figure(figsize=(20, 20))
plt.scatter(filter[xas], filter[yas], c=filter[colorbar], cmap='viridis')
plt.title('GPS data vs speed')
plt.xlabel('Latitude')
plt.ylabel('Longitude')

# Adding a colorbar with a label for the speed in km/h
cbar = plt.colorbar()
cbar.set_label('Speed (km/h)')

# Coordinates for the box
lat_min, lat_max = 2.445, 2.455
lon_min, lon_max = 0.28+4.346e3,0.31+4.346e3 
# Plotting the box
plt.plot([lat_min, lat_max, lat_max, lat_min, lat_min],
         [lon_min, lon_min, lon_max, lon_max, lon_min],
         color='red', linestyle='--', linewidth=2)

plt.grid()
plt.show()

In [242]:
lat_min, lat_max = 2.445, 2.455
lon_min, lon_max = 0.29+4.346e3,0.305+4.346e3 
yas = 'Latitude'
xas = 'Longitude'

# Maak een booleaanse masker voor de condities
in_square = (lat_min <= filter[yas]) & (filter[yas] <= lat_max) & (lon_min <= filter[xas]) & (filter[xas] <= lon_max)

# Gebruik het masker om te zien welke rijen erin zitten
if in_square.any():
    print("Er zijn punten binnen het vierkant.")
else:
    print("Geen punten binnen het vierkant.")
    
# Optioneel: print de rijen die binnen het vierkant vallen
punten_binnen = filter[in_square]
print(punten_binnen)

Geen punten binnen het vierkant.
Empty DataFrame
Columns: [Datalogger port, Dataloggertijd, in s, Format header (>08|03), NMEA type($GPRMC), Tijd, Status, Latitude, Noord of Zuid (N/S), Longitude, Oost of West (E/W), Snelheid over de grond in km/h, Richting van de snelheid, Datum, Magnetische variatie, Magnetische directie, Mode indicator (A), Data in balancers, DegreesTrue, T, DegreesMagnetic, M, Snelheid tenopzichte van het water in Knots, N, Snelheid tenopzichte van het water in Kilometers, K]
Index: []

[0 rows x 25 columns]
